# Acquire and Prepare

---

This notebook contains all steps taken in the data acquisition and preparation phases of the data science pipeline for the Zillow clustering project. This notebook does rely on helper files so if you want to run the code blocks in this notebook ensure that you have all the helper files in the same directory.

---

## The Required Imports

As stated before this notebook relies on some helper files which are imported below. This notebook also relies on numpy, pandas, matplotlib, seaborn, and sklearn.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from acquire import AcquireZillow

---

## Data Acquisition

Here we will acquire the data from the zillow database hosted at data.codeup.com. We are acquiring properties with transaction dates in 2017. Additionally, for any properties with more than one transaction in 2017 we are only acquiring the most recent transaction. Finally, we are acquiring only single unit/single family homes.

In [2]:
# All the code needed to acquire the data is encapsulated in the acquire.py file using the following parameters.
#
# Cache file: zillow.csv
# Database name: zillow
# Hostname: data.codeup.com
# 
# SQL Query:
#
#         SELECT
#             properties_2017.*,
#             logerror,
#             transactiondate,
#             typeconstructiondesc,
#             airconditioningdesc,
#             architecturalstyledesc,
#             buildingclassdesc,
#             propertylandusedesc,
#             storydesc,
#             heatingorsystemdesc
#         FROM properties_2017
#         JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
#             AND predictions_2017.transactiondate LIKE '2017%%'
#         LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
#         LEFT JOIN airconditioningtype USING (airconditioningtypeid)
#         LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
#         LEFT JOIN buildingclasstype USING (buildingclasstypeid)
#         LEFT JOIN propertylandusetype USING (propertylandusetypeid)
#         LEFT JOIN storytype USING (storytypeid)
#         LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)

#         JOIN (
#             SELECT
#                 parcelid,
#                 MAX(transactiondate) AS date
#             FROM predictions_2017
#             GROUP BY parcelid
#         ) AS max_dates ON properties_2017.parcelid = max_dates.parcelid
#             AND predictions_2017.transactiondate = max_dates.date

#         WHERE latitude IS NOT NULL AND longitude IS NOT NULL;

zillow = AcquireZillow().get_data()

In [4]:
# How many observations do we have? 77380
zillow.shape

(77380, 72)

In [5]:
# Let's see what columns we can drop before moving on to preparation.
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 72 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         49439 non-null  float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77380 non-null  float64
 3   buildingclasstypeid           15 non-null     float64
 4   architecturalstyletypeid      206 non-null    float64
 5   airconditioningtypeid         24953 non-null  float64
 6   typeconstructiontypeid        222 non-null    float64
 7   id                            77380 non-null  int64  
 8   parcelid                      77380 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77380 non-null  float64
 11  bedroomcnt                    77380 non-null  float64
 12  buildingqualitytypeid         49671 non-null  float64
 13  c

In [6]:
# Let's drop all the foreign key id columns, those will be useless to us.
drop_columns = [
    'heatingorsystemtypeid',
    'storytypeid',
    'propertylandusetypeid',
    'buildingclasstypeid',
    'architecturalstyletypeid',
    'airconditioningtypeid',
    'typeconstructiontypeid',
    'id',
    'parcelid'
]

zillow = zillow.drop(columns = drop_columns)
zillow.shape

(77380, 60)

In [7]:
# Let's make sure it worked.
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   basementsqft                  50 non-null     float64
 1   bathroomcnt                   77380 non-null  float64
 2   bedroomcnt                    77380 non-null  float64
 3   buildingqualitytypeid         49671 non-null  float64
 4   calculatedbathnbr             76771 non-null  float64
 5   decktypeid                    614 non-null    float64
 6   finishedfloor1squarefeet      6023 non-null   float64
 7   calculatedfinishedsquarefeet  77184 non-null  float64
 8   finishedsquarefeet12          73748 non-null  float64
 9   finishedsquarefeet13          41 non-null     float64
 10  finishedsquarefeet15          3009 non-null   float64
 11  finishedsquarefeet50          6023 non-null   float64
 12  finishedsquarefeet6           386 non-null    float64
 13  f

Everything looks good. Let's move on to preparation.

---

## Data Preparation

Now we will begin preparing the data. The goal here is to handle all missing values and split the data to prevent data leakage.